In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
batch_size = 32
img_h = 180
img_w = 180
epochs = 100

In [4]:
train_df = tf.keras.utils.image_dataset_from_directory(
    "cow",
    validation_split = 0.2,
    subset= 'training',
    seed= 123,
    image_size = (img_h, img_w),
    batch_size = batch_size
)

Found 55 files belonging to 1 classes.
Using 44 files for training.


In [5]:
valid_df = tf.keras.utils.image_dataset_from_directory(
    "cow",
    validation_split = 0.2,
    subset = "validation",
    seed=123,
    image_size = (img_h,img_w),
    batch_size = batch_size
)

Found 55 files belonging to 1 classes.
Using 11 files for validation.


In [6]:
data_aug =  tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.4),
    layers.RandomZoom(0.3),
    layers.RandomContrast(0.1),
    layers.RandomBrightness(0.1)
])

In [7]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet',input_shape=(img_h,img_w,3),include_top=False)

base_model.trainable= False

C:\Users\Dell\AppData\Local\Temp\ipykernel_11512\945699244.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet',input_shape=(img_h,img_w,3),include_top=False)


In [15]:
num_cl = len(train_df.class_names)

model = Sequential(
    [ 
        data_aug,
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(64,activation='relu'),
        layers.Dropout(0.25),
        layers.Dense(num_cl,activation='sigmoid')
    ]
)

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
            #   loss=tf.keras.losses.binarycrossentropy(),
            loss='binary_crossentropy',
              metrics= ['accuracy'])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)


In [21]:
his = model.fit(train_df, validation_data=valid_df, epochs=20, callbacks=[early, lr_reduce])


Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 48s 8s/step - accuracy: 0.4318 - loss: 0.9256 - val_accuracy: 1.0000 - val_loss: 0.0014 - learning_rate: 0.0020
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 843ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 1.6674e-05 - learning_rate: 0.0020
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 930ms/step - accuracy: 1.0000 - loss: 8.9417e-05 - val_accuracy: 1.0000 - val_loss: 5.5352e-07 - learning_rate: 0.0020
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 4.5601e-06 - val_accuracy: 1.0000 - val_loss: 3.5540e-08 - learning_rate: 0.0020
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 864ms/step - accuracy: 1.0000 - loss: 1.4394e-07 - val_accuracy: 1.0000 - val_loss: 2.2989e-08 - learning_rate: 4.0000e-04
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 999ms/step - accuracy: 1.0000 - loss: 1.3040e-07 - val_accuracy: 1.0000 - val_loss: 1.6193e-08 - learning_rate: 4.0000e-04
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 886ms/step - accuracy:

In [ ]:
base_model.trainable = True

# Freeze most layers — only keep last 50 trainable
for layer in base_model.layers[:-50]:
    layer.trainable = False


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
history2 = model.fit(train_df, validation_data=valid_df, epochs=20, callbacks=[early, lr_reduce])



In [ ]:
base_model.trainable = True

# Freeze most layers — only keep last 50 trainable
for layer in base_model.layers[:110]:
    layer.trainable = False


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-6),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

history3 = model.fit(train_df, validation_data=valid_df, epochs=20, callbacks=[early, lr_reduce])

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

class_names = train_df.class_names

def prep_img(imf_path,img_h,img_w):
  img= image.load_img(img_path,target_size=(img_h,img_w))
  img_arr = image.img_to_array(img)
  img_arr = np.expand_dims(img_arr,axis=0)
  return img_arr
# C:\Users\Dell\Documents\python projects\deep tech\boat data\train\20130412_122808_99204.jpg
# C:\Users\Dell\Documents\python projects\deep tech\boat data\train\20130412_122352_97982.jpg
img_path = "boat data/train/VaporettoACTV2.jpeg"
img_arr = prep_img(img_path,img_h,img_w)
prediction = model.predict(img_arr)
# prediction
pred_class_idx = np.argmax(prediction, axis=1)[0]
pred_class_name = class_names[pred_class_idx]
pred_class_name
